In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [18]:
df=pd.read_csv("../data/processed/ward_flood_features1.csv")
df.describe()

,avg_rainfall,max_rainfall,waterlogging_count,avg_severity,max_severity
count,24.000000,24.000000,24.000000,24.000000,24.000000
mean,8.767905,188.204439,8.166667,1.150330,1.833333
std,1.113114,32.262572,3.292504,0.255338,0.380693
min,6.227481,148.573600,3.000000,0.666667,1.000000
25%,8.415042,165.252910,6.000000,1.000000,2.000000
50%,8.415042,165.252910,7.000000,1.142857,2.000000
75%,9.098397,212.424270,9.500000,1.258929,2.000000
max,10.600971,240.544700,15.000000,1.750000,2.000000


In [31]:
df["risk_score"] = (
    0.4 * df["avg_rainfall"] +
    0.3* df["max_rainfall"] +
    0.2* df["waterlogging_count"] +
    0.1 * df["avg_severity"]
)

In [32]:
df["risk_score"] = (
    df["risk_score"] - df["risk_score"].min()
) / (
    df["risk_score"].max() - df["risk_score"].min()
)

In [33]:
features = [
    "avg_rainfall",
    "max_rainfall",
    "waterlogging_count",
    "avg_severity",
    "max_severity"
]

X = df[features].values
y = df["risk_score"].values

In [34]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [35]:
model = Sequential([
    Dense(32, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(16, activation="relu"),
    Dense(1, activation="linear")   # regression output
])

c:\Users\Aryan\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["mae"]
)

In [37]:
history = model.fit(
    X_train,
    y_train,
    epochs=200,
    batch_size=4,
    validation_split=0.2,
    verbose=1
)

Epoch 1/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 0.1912 - mae: 0.3449 - val_loss: 0.0620 - val_mae: 0.1976
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1358 - mae: 0.2888 - val_loss: 0.0473 - val_mae: 0.1773
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0955 - mae: 0.2454 - val_loss: 0.0376 - val_mae: 0.1623
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0676 - mae: 0.2077 - val_loss: 0.0311 - val_mae: 0.1486
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0464 - mae: 0.1788 - val_loss: 0.0278 - val_mae: 0.1349
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0318 - mae: 0.1500 - val_loss: 0.0285 - val_mae: 0.1241
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0248 - mae: 0.1313 - val_loss: 0.0318 - val_mae: 0.1121
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0168 - mae: 0.1113 - val_loss: 0.0353 - val_mae: 0.1186
Epoch 9/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0147 - mae: 

In [38]:
loss, mae = model.evaluate(X_test, y_test)
print("Test MAE:", mae)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - loss: 0.0123 - mae: 0.0635
Test MAE: 0.0635041743516922


In [39]:
df["predicted_risk_score"] = model.predict(
    scaler.transform(X)
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step


In [40]:
def risk_zone(score):
    if score < 0.33:
        return "Low"
    elif score < 0.66:
        return "Medium"
    else:
        return "High"

df["risk_zone"] = df["predicted_risk_score"].apply(risk_zone)

In [42]:
df.to_csv(
    "../data/processed/ward_flood_risk_nn.csv",
    index=False
)